<a href="https://colab.research.google.com/github/royam0820/fastai2-v4/blob/master/Camembert_LM_TextClas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

     |████████████████████████████████| 204kB 4.1MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 


In [ ]:
!pip install -Uq transformers
from fastai.text.all import *

     |████████████████████████████████| 2.3MB 4.2MB/s 
     |████████████████████████████████| 3.3MB 50.9MB/s 
     |████████████████████████████████| 901kB 42.9MB/s 


In [ ]:
# # better display of review text in dataframes
# pd.set_option('display.max_colwidth', None) 

# CamentBERT

In [ ]:
!pip install sentencepiece

from transformers import CamembertModel, CamembertTokenizer

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base")

     |████████████████████████████████| 1.2MB 4.1MB/s 


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
ids = tokenizer.encode('This is an example of text, and')
ids

[5, 17526, 2856, 674, 1017, 21598, 786, 21, 12584, 7, 1168, 6]

In [ ]:
model.eval()

CamembertModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dr

# Getting the data

In [ ]:
# Download repo and its dependencies 
!git clone https://github.com/TheophileBlard/french-sentiment-analysis-with-bert/
!cd french-sentiment-analysis-with-bert

Cloning into 'french-sentiment-analysis-with-bert'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 383 (delta 91), reused 101 (delta 30), pack-reused 182
Receiving objects: 100% (383/383), 163.92 MiB | 23.43 MiB/s, done.
Resolving deltas: 100% (154/154), done.


In [ ]:
!ls

french-sentiment-analysis-with-bert  sample_data


In [ ]:
!tar -xf /content/french-sentiment-analysis-with-bert/allocine_dataset/data.tar.bz2

In [ ]:
!cd data; ls

allocine_dataset.pickle  test.jsonl  train.jsonl  val.jsonl


## Training Dataset

In [ ]:
train_df = pd.read_json('/content/data/train.jsonl', lines=True, nrows=600)
train_df.head(1)

,film-url,review,polarity
0,http://www.allocine.fr/film/fichefilm-135259/critiques/spectateurs,"Si vous cherchez du cinéma abrutissant à tous les étages,n'ayant aucune peur du cliché en castagnettes et moralement douteux,""From Paris with love"" est fait pour vous.Toutes les productions Besson,via sa filière EuropaCorp ont de quoi faire naître la moquerie.Paris y est encore une fois montrée comme une capitale exotique,mais attention si l'on se dirige vers la banlieue,on y trouve tout plein d'intégristes musulmans prêts à faire sauter le caisson d'une ambassadrice américaine.Nauséeux.Alors on se dit qu'on va au moins pouvoir apprécier la déconnade d'un classique buddy-movie avec le jeun...",0


In [ ]:
train_df.shape

(600, 3)

In [ ]:
some_review = train_df.review[0]; some_review

'Si vous cherchez du cinéma abrutissant à tous les étages,n\'ayant aucune peur du cliché en castagnettes et moralement douteux,"From Paris with love" est fait pour vous.Toutes les productions Besson,via sa filière EuropaCorp ont de quoi faire naître la moquerie.Paris y est encore une fois montrée comme une capitale exotique,mais attention si l\'on se dirige vers la banlieue,on y trouve tout plein d\'intégristes musulmans prêts à faire sauter le caisson d\'une ambassadrice américaine.Nauséeux.Alors on se dit qu\'on va au moins pouvoir apprécier la déconnade d\'un classique buddy-movie avec le jeune agent aux dents longues obligé de faire équipe avec un vieux lou complètement timbré.Mais d\'un côté,on a un Jonathan Rhys-meyers fayot au possible,et de l\'autre un John Travolta en total délire narcissico-badass,crâne rasé et bouc proéminent à l\'appui.Sinon,il n\'y a aucun scénario.Seulement,des poursuites débiles sur l\'autoroute,Travolta qui étale 10 mecs à l\'arme blanche en 8 mouvement

In [ ]:
tokenizer.tokenize(some_review)[:15]

['▁Si',
 '▁vous',
 '▁cherchez',
 '▁du',
 '▁cinéma',
 '▁abruti',
 'ssant',
 '▁à',
 '▁tous',
 '▁les',
 '▁étages',
 ',',
 'n',
 "'",
 'ayant']

In [ ]:
tokenizer.encode(some_review)[:15]

[5, 168, 39, 3162, 25, 1545, 29470, 2927, 15, 117, 19, 9339, 7, 255, 11]

In [ ]:
tokenizer.decode(tokenizer.encode(some_review))

'<s> Si vous cherchez du cinéma abrutissant à tous les étages,n\'ayant aucune peur du cliché en castagnettes et moralement douteux,"From Paris with love" est fait pour vous.Toutes les productions Besson,via sa filière EuropaCorp ont de quoi faire naître la moquerie.Paris y est encore une fois montrée comme une capitale exotique,mais attention si l\'on se dirige vers la banlieue,on y trouve tout plein d\'intégristes musulmans prêts à faire sauter le caisson d\'une ambassadrice américaine.Nauséeux.Alors on se dit qu\'on va au moins pouvoir apprécier la déconnade d\'un classique buddy-movie avec le jeune agent aux dents longues obligé de faire équipe avec un vieux lou complètement timbré.Mais d\'un côté,on a un Jonathan Rhys-meyers fayot au possible,et de l\'autre un John Travolta en total délire narcissico-badass,crâne rasé et bouc proéminent à l\'appui.Sinon,il n\'y a aucun scénario.Seulement,des poursuites débiles sur l\'autoroute,Travolta qui étale 10 mecs à l\'arme blanche en 8 mouve

In [ ]:
train_df.to_csv('/content/data/train.csv', encoding = 'utf-8', header = False, index = False)

In [ ]:
df_train = pd.read_csv('/content/data/train.csv', encoding = 'utf-8', header=None)  

In [ ]:
# !cp /content/data/train.csv /content/drive/MyDrive

## Validation Set

In [ ]:
val_df = pd.read_json('/content/data/val.jsonl', lines=True, nrows=200)
val_df.head(1)

,film-url,review,polarity
0,http://www.allocine.fr/film/fichefilm-51895/critiques/spectateurs,"Ce film est tout ce qu'il y a de plus sympa. Même si l'ensemble n'est pas dépourvu de clichés, il serait hypocrite de dire que ce film est ennuyeux à regarder, bien au contraire. Il est très plaisant à regarder et même si l'ensemble est asse convenue, la mise en scène de Peter Chelsom est légère et ce film est une sorte de bouffée d'air frais. Tout à fait estimable.",0


In [ ]:
val_df.shape

(200, 3)

In [ ]:
val_df.to_csv('/content/data/val.csv', encoding = 'utf-8', header = False, index = False)

In [ ]:
df_valid = pd.read_csv('/content/data/val.csv', encoding = 'utf-8', header=None)

In [ ]:
!cp /content/data/val.csv /content/drive/MyDrive

cp: cannot create regular file '/content/drive/MyDrive': No such file or directory


## Train and Validation Datasets

In [ ]:
all_texts = np.concatenate([df_train[1].values, df_valid[1].values])

In [ ]:
# print(train_df['polarity'].unique(), val_df['polarity'].unique())

NOTE: polarity: `0` negative review; `1` positive review

In [ ]:
print(len(train_df), len(val_df), len(all_texts))

600 200 800


In [ ]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [ ]:
splits = [range_of(df_train), list(range(len(df_train), len(all_texts)))]
tls = TfmdLists(all_texts, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)

In [ ]:
# tls.train[0],tls.valid[0]

In [ ]:
tls.tfms(tls.train.items[0]).shape, tls.tfms(tls.valid.items[0]).shape

(torch.Size([332]), torch.Size([93]))

In [ ]:
show_at(tls.train, 0)

Si vous cherchez du cinéma abrutissant à tous les étages,n'ayant aucune peur du cliché en castagnettes et moralement douteux,"From Paris with love" est fait pour vous.Toutes les productions Besson,via sa filière EuropaCorp ont de quoi faire naître la moquerie.Paris y est encore une fois montrée comme une capitale exotique,mais attention si l'on se dirige vers la banlieue,on y trouve tout plein d'intégristes musulmans prêts à faire sauter le caisson d'une ambassadrice américaine.Nauséeux.Alors on se dit qu'on va au moins pouvoir apprécier la déconnade d'un classique buddy-movie avec le jeune agent aux dents longues obligé de faire équipe avec un vieux lou complètement timbré.Mais d'un côté,on a un Jonathan Rhys-meyers fayot au possible,et de l'autre un John Travolta en total délire narcissico-badass,crâne rasé et bouc proéminent à l'appui.Sinon,il n'y a aucun scénario.Seulement,des poursuites débiles sur l'autoroute,Travolta qui étale 10 mecs à l'arme blanche en 8 mouvements(!!)ou laiss

In [ ]:
show_at(tls.valid, 0)

Ce film est tout ce qu'il y a de plus sympa. Même si l'ensemble n'est pas dépourvu de clichés, il serait hypocrite de dire que ce film est ennuyeux à regarder, bien au contraire. Il est très plaisant à regarder et même si l'ensemble est asse convenue, la mise en scène de Peter Chelsom est légère et ce film est une sorte de bouffée d'air frais. Tout à fait estimable.


In [ ]:
#bs,sl = 4,256 # not working
bs,sl = 16, 512
dls_lm = tls.dataloaders(bs=bs, seq_len=sl)

In [ ]:
# dls.one_batch() 

In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"Malgré une bande annonce prometteuse, You're Next est un pur nanar se rêvant mieux foutu qu'il ne l'est vraiment, cumulant tous les clichés en la matière quand il ne s'agit pas d'incohérences absolument ridicules (la mère qui se fait tuer dans la chambre fermée de l'intérieur et personne pour comprendre que le tueur est nécessairement dans la pièce!!) Circulez y a rien à voir! Un très bon film. Une très comédie politique. Tim Robbins incarne un candidat à la maison blanche des États Unis. BOF BOF BOF!!! pas emballé du tout apres 10 mn de film... jolie scene de combat mais qui n'arriveront pas a me faire relever la note.. A passer. Pour moi, un film qui ne sert à rien. Les personnages sont tous plus stupides les uns que les autres, arrive en tête le personnage de Elise interprété par une Judith Godrèche pas très inspiré avec une voix","une bande annonce prometteuse, You're Next est un pur nanar se rêvant mieux foutu qu'il ne l'est vraiment, cumulant tous les clichés en la matière quand il ne s'agit pas d'incohérences absolument ridicules (la mère qui se fait tuer dans la chambre fermée de l'intérieur et personne pour comprendre que le tueur est nécessairement dans la pièce!!) Circulez y a rien à voir! Un très bon film. Une très comédie politique. Tim Robbins incarne un candidat à la maison blanche des États Unis. BOF BOF BOF!!! pas emballé du tout apres 10 mn de film... jolie scene de combat mais qui n'arriveront pas a me faire relever la note.. A passer. Pour moi, un film qui ne sert à rien. Les personnages sont tous plus stupides les uns que les autres, arrive en tête le personnage de Elise interprété par une Judith Godrèche pas très inspiré avec une voix très"
1,"succession de séquences ridicules et rarement drôles, dommage. • Tout d’abord bravo à toute l'équipe qui a réaliser ce très beau reportage!! • Ce film a vraiment le mérite d’être vu et revu, Il peut se regarder en famille tout • ages confondus et même sans être grand fan de basket car il montre et donne • de belles leçons dans tous domaines et pas seulement au niveau sportif!!! • Je vous encourage donc tous à l'acheter, le regarder et même l’offrir vous ne le regretterez vraiment pas.... • Evan a énormément de mérite et de courage il est vraiment un bel exemple, je lui souhaite d’aller encore bien plus loin et franchement je pense que l’on a pas fini d’entendre parler de lui....... • Alors dépêchez-vous d’aller acheter le DVD EVAN ""le rêve américain"" qui est en vente en boutique pour la maudite somme de 15€, et en même","de séquences ridicules et rarement drôles, dommage. • Tout d’abord bravo à toute l'équipe qui a réaliser ce très beau reportage!! • Ce film a vraiment le mérite d’être vu et revu, Il peut se regarder en famille tout • ages confondus et même sans être grand fan de basket car il montre et donne • de belles leçons dans tous domaines et pas seulement au niveau sportif!!! • Je vous encourage donc tous à l'acheter, le regarder et même l’offrir vous ne le regretterez vraiment pas.... • Evan a énormément de mérite et de courage il est vraiment un bel exemple, je lui souhaite d’aller encore bien plus loin et franchement je pense que l’on a pas fini d’entendre parler de lui....... • Alors dépêchez-vous d’aller acheter le DVD EVAN ""le rêve américain"" qui est en vente en boutique pour la maudite somme de 15€, et en même temps"


In [ ]:
def tokenize(text):
    toks = tokenizer.tokenize(text)
    return tensor(tokenizer.convert_tokens_to_ids(toks))

tokenized = [tokenize(t) for t in progress_bar(all_texts)]

In [ ]:
len(tokenized)

800

In [ ]:
tokenizer.vocab_size

32005

In [ ]:
vocab_lm = tokenizer.save_vocabulary('/content/data/')

NOTE: vocab file /content/data/sentencepiece.bpe.model

It's probably because your token embeddings size (vocab size) doesn't match with pre-trained model. Do model.resize_token_embeddings(len(tokenizer)) before training.

In [ ]:
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
# reviews_len = [len(tokenizer.encode(df_train, max_length=512))
#                           for review in df_train]

In [ ]:
# MAX_SEQ_LEN = 400 # in terms of generated tokens (not words)

# short_reviews = sum(np.array(reviews_len) <= MAX_SEQ_LEN)
# long_reviews = sum(np.array(reviews_len) > MAX_SEQ_LEN)

# print("{} reviews with LEN > {} ({:.2f} % of total data)".format(
#     long_reviews,
#     MAX_SEQ_LEN,
#     100 * long_reviews / len(reviews_len)
# ))

In [ ]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        return x if isinstance(x, Tensor) else tokenize(x)
        
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [ ]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [ ]:
# del learn

In [ ]:
learn = Learner(dls_lm, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/autocast_mode.py:118: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


In [ ]:
learn.model

CamembertModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dr

In [ ]:
learn.validate()

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:116: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


IndexError: ignored

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, 1e-4)

In [ ]:
df_valid.head(1)

In [ ]:
prompt = "Voici un gros chat "

In [ ]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
inp.shape

In [ ]:
preds = learn.model.generate(inp, max_length=40, num_beams=5, temperature=1.5)

In [ ]:
tokenizer.decode(preds[0].cpu().numpy())